<img src="img/pandora2d_logo.png" width="500" height="500">

# Pandora2D : a coregistration framework

# Introduction and basic usage

#### Imports and external functions

In [ ]:
import io
from pathlib import Path
from IPython.display import Image, display
from pprint import pprint
import numpy as np

In [ ]:
from snippets.utils import *

## Pandora2D's pipeline

Pandora2D provides the following steps:
* estimation computation
* matching cost computation
* disparity computation (**mandatory if matching_cost**)
* subpixel disparity refinement

<img src="img/Pandora2D_pipeline.drawio.svg" width="1000">

## About the estimation step :

1. used alone, it computes a constant a priori offset across the entire image in line and column. 

2. When used in a complete pipeline, it enables, **thanks to an interval entered by the user**, the calculation of "col_disparity" and "row_disparity" parameters in the input configuration dictionnary.

The outputs are : 
- shown in log in verbose mode
- saved in the configuration file
- stored in dataset image

# Pandora2D execution options with state machine

#### Imports of pandora2d

In [ ]:
# Load pandora2d imports
import pandora2d
from pandora2d.state_machine import Pandora2DMachine
from pandora2d.check_configuration import check_conf
from pandora2d.img_tools import create_datasets_from_inputs

#### Load and visualize input data 

Provide image path

In [ ]:
# Paths to left and right images
img_left_path = "data/left.tif"
img_right_path = "data/right.tif"

#### No need for disparity Range in the estimation process.

In [ ]:
user_cfg = {
    "input": {
        "left": {
            "img": img_left_path, 
            "nodata": np.nan
        },
        "right": {
            "img": img_right_path, 
            "nodata": np.nan
        },
    },
    "pipeline": { 
        "estimation": {
            "estimation_method": "phase_cross_correlation",
            "range_row": 5, 
            "range_col": 5,
            "sample_factor": 100
        },
    },
    "output": {
        "path": "outputs/estimation_step_explained"
    },
}

#### Instantiate the machine

In [ ]:
pandora2d_machine = Pandora2DMachine()

#### Check the configuration and sequence of steps

In [ ]:
checked_cfg = check_conf(user_cfg, pandora2d_machine)

#### Create image datasets

In [ ]:
img_left, img_right = create_datasets_from_inputs(input_config=user_cfg["input"], estimation_cfg=user_cfg["pipeline"].get("estimation"))

#### Trigger all the steps of the machine at ones

In [ ]:
dataset, estimation_results = pandora2d.run(
    pandora2d_machine,
    img_left,
    img_right,
    checked_cfg
    )

## The results are added to a configuration dictionnary 

##### Disparities ranges are set

In [ ]:
print("Disparity range for columns : ", estimation_results["input"]["col_disparity"])
print("Disparity range for rows : ", estimation_results["input"]["row_disparity"])

##### And many informations are stored in estimation dictionnary configuration

In [ ]:
estimation_results["pipeline"]["estimation"]